In [1]:
#| default_exp interface_pymeshlab

In [2]:
#| export

from blender_tissue_cartography import io as tcio

In [3]:
#| export

import numpy as np
import os
from copy import deepcopy
import pymeshlab

Unable to load the following plugins:

	libio_e57.so: libio_e57.so does not seem to be a Qt Plugin.

Cannot load library /home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/pymeshlab/lib/plugins/libio_e57.so: (/lib/x86_64-linux-gnu/libp11-kit.so.0: undefined symbol: ffi_type_pointer, version LIBFFI_BASE_7.0)



In [4]:
import matplotlib.pyplot as plt
from importlib import reload

## Interfacing with `pymeshlab`

In this notebook, we define functions to convert our `ObjMesh` class to and from `pymeshlab`.

In [5]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"

In [6]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

In [20]:
def to_pymeshlab(mesh: tcio.ObjMesh):
    """Convert tcio.ObjMesh to pymeshlab.Mesh"""
    mesh.match_vertex_info(require_texture_normals=False)
    return pymeshlab.Mesh(vertex_matrix=mesh.matched_vertices,
                          face_list_of_indices=[np.array([v[0] for v in fc]) for fc in mesh.faces],
                          v_normals_matrix=mesh.matched_normals,
                          v_tex_coords_matrix=mesh.matched_texture_vertices,)

In [22]:
to_pymeshlab(mesh_ref)

In [18]:
mesh_ref.match_vertex_info(require_texture_normals=False)
pymeshlab.Mesh(vertex_matrix=mesh_ref.matched_vertices,
               face_list_of_indices=[np.array([v[0] for v in fc]) for fc in mesh_ref.faces],
               v_normals_matrix=mesh_ref.matched_normals,
               v_tex_coords_matrix=mesh_ref.matched_texture_vertices,
              )

In [19]:
mesh_ref.match_vertex_info(require_texture_normals=False)
pymeshlab.Mesh(vertex_matrix=mesh_ref.matched_vertices,
               face_list_of_indices=mesh_ref.tris,
               v_normals_matrix=mesh_ref.matched_normals,
               v_tex_coords_matrix=mesh_ref.matched_texture_vertices,
              )